In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb

## NOTE: this model is gonna be bad... not symmetrical, inputs arbitrarily chosen, not enough training data

# reading dat
dat = pd.read_csv("MLData/combined.csv")

# modifying dat to remove character values
dat["WinAsInt"] = dat["B_Win"].transform(lambda x: int(x == "WIN"))
dat = dat.drop(axis = 1, labels = "B_Win")

In [4]:
correlations = dat.corr()["WinAsInt"].sort_values(ascending = False)

print(correlations)

WinAsInt         1.000000
B_TOP_GPM        0.181217
R_BOT_Deaths     0.166334
B_BOT_Assists    0.154807
B_BOT_KAPM       0.151259
                   ...   
R_JNG_GPM       -0.157942
R_MID_KAPM      -0.167307
R_WinNum        -0.170781
R_MID_GD15      -0.172071
R_Kills         -0.183026
Name: WinAsInt, Length: 153, dtype: float64


In [5]:
# thinning out dataset
print(correlations[correlations < -0.13])
print()
print(correlations[correlations > 0.13])

R_MID_Assists   -0.133319
R_JNG_CSM       -0.135401
R_JNG_Kills     -0.137047
R_Mid_Plates    -0.140487
R_BOT_GPM       -0.142660
R_Barons        -0.147552
R_MID_Kills     -0.147868
R_Towers        -0.148539
R_JNG_KAPM      -0.148710
R_JNG_GPM       -0.157942
R_MID_KAPM      -0.167307
R_WinNum        -0.170781
R_MID_GD15      -0.172071
R_Kills         -0.183026
Name: WinAsInt, dtype: float64

WinAsInt         1.000000
B_TOP_GPM        0.181217
R_BOT_Deaths     0.166334
B_BOT_Assists    0.154807
B_BOT_KAPM       0.151259
B_TOP_Kills      0.146534
B_JNG_KAPM       0.145647
B_Kills          0.143275
B_BOT_XPD15      0.139848
B_WinNum         0.137614
B_MID_VSPM       0.135714
B_TOP_CSM        0.133283
B_BOT_CSD15      0.130068
Name: WinAsInt, dtype: float64


In [2]:
input_vars = correlations[correlations < -0.13].index.to_list() + correlations[correlations > 0.13].index.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)


NameError: name 'correlations' is not defined

In [44]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 10
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-auc:0.68431	test-auc:nan
[1]	train-auc:0.69688	test-auc:nan
[2]	train-auc:0.73390	test-auc:nan
[3]	train-auc:0.74110	test-auc:nan
[4]	train-auc:0.76009	test-auc:nan
[5]	train-auc:0.76444	test-auc:nan
[6]	train-auc:0.76840	test-auc:nan
[7]	train-auc:0.77253	test-auc:nan
[8]	train-auc:0.77553	test-auc:nan
[9]	train-auc:0.77488	test-auc:nan


c:\Users\carte\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\carte\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:19:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\metric\auc.cc:320: Dataset is empty, or contains only positive or negative samples.
  warnings.warn(smsg, UserWarning)


In [45]:
#xgb.plot_importance(bst)

In [46]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Small Model Scores (0.13)"] = scores

print(teams)

           B_Team         R_Team  Small Model Scores (0.13)
0   Team Vitality  Team Heretics                   0.633670
1          Fnatic          Rogue                   0.651725
2        Team BDS     G2 Esports                   0.640879
3    Karmine Corp         GIANTX                   0.586876
4   MAD Lions KOI      SK Gaming                   0.668630
5   Team Vitality       Team BDS                   0.526131
6   Team Heretics          Rogue                   0.628600
7          Fnatic   Karmine Corp                   0.628128
8          GIANTX      SK Gaming                   0.611577
9   MAD Lions KOI     G2 Esports                   0.518184
10          Rogue       Team BDS                   0.447782
11      SK Gaming  Team Vitality                   0.549669
12         GIANTX  MAD Lions KOI                   0.504997
13  Team Heretics         Fnatic                   0.537114
14     G2 Esports   Karmine Corp                   0.628128
15      SK Gaming       Team BDS        

In [47]:
input_vars = correlations[correlations < -0.10].index.to_list() + correlations[correlations > 0.10].index.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

      R_SUP_CS   R_MID_GPM  B_TOP_Deaths  B_BOT_VSPM  R_TOP_KAPM  R_MID_CSD15  \
0    34.894737  395.105263           NaN         NaN    0.208421     7.789474   
1    32.421053  389.250000      2.684211         NaN         NaN     3.300000   
2          NaN         NaN      1.666667    1.120000    0.240556          NaN   
3    39.090909  417.666667      2.888889    1.221053    0.258889    -3.111111   
4    30.647059  375.470588           NaN         NaN         NaN    -4.588235   
..         ...         ...           ...         ...         ...          ...   
361  37.700000  384.100000      3.222222    1.533333    0.201000     3.100000   
362  23.333333         NaN      2.444444    1.645556    0.154444          NaN   
363  29.333333  433.000000      2.555556    1.326667    0.245556    19.111111   
364  30.111111  421.555556      3.700000         NaN    0.396667    -2.444444   
365  27.111111  392.000000      2.666667    1.557778    0.240000    -3.555556   

     R_TOP_Assists  R_BOT_K

In [1]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 30
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

NameError: name 'dtrain' is not defined

In [49]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
# teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Med Model Scores (0.1)"] = scores

print(teams)

           B_Team         R_Team  Small Model Scores (0.13)  \
0   Team Vitality  Team Heretics                   0.633670   
1          Fnatic          Rogue                   0.651725   
2        Team BDS     G2 Esports                   0.640879   
3    Karmine Corp         GIANTX                   0.586876   
4   MAD Lions KOI      SK Gaming                   0.668630   
5   Team Vitality       Team BDS                   0.526131   
6   Team Heretics          Rogue                   0.628600   
7          Fnatic   Karmine Corp                   0.628128   
8          GIANTX      SK Gaming                   0.611577   
9   MAD Lions KOI     G2 Esports                   0.518184   
10          Rogue       Team BDS                   0.447782   
11      SK Gaming  Team Vitality                   0.549669   
12         GIANTX  MAD Lions KOI                   0.504997   
13  Team Heretics         Fnatic                   0.537114   
14     G2 Esports   Karmine Corp                   0.62

In [50]:
input_vars = dat.columns.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

       B_Kills  B_Towers  B_Drakes  B_Barons  B_Plates  B_Top_Plates  \
0    13.555556  7.444444  2.611111  0.944444  5.055556      2.666667   
1    11.684211  6.263158  2.105263  0.894737  6.000000      2.263158   
2    15.666667  7.444444  3.222222  0.944444  4.388889      1.444444   
3    10.388889  5.222222  1.944444  0.500000  4.250000      1.187500   
4    10.611111  6.000000  2.722222  0.611111  4.117647      0.941176   
..         ...       ...       ...       ...       ...           ...   
361  15.000000  8.222222  3.111111  1.444444  6.222222      2.111111   
362  16.777778  8.222222  2.888889  0.888889  5.666667      2.000000   
363  10.333333  6.111111  1.555556  0.666667  4.222222      1.777778   
364  17.200000  4.800000  2.300000  0.700000  5.000000      2.000000   
365  11.888889  5.444444  2.000000  0.555556  3.444444      1.222222   

     B_Mid_Plates  B_Bot_Plates  B_Game_Time_s  B_Gold_Num  ...  R_SUP_CSM  \
0        0.833333      1.555556    1909.055556   60.48333

(366, 152)
(0, 152)


In [51]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 30
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-auc:0.69237	test-auc:nan
[1]	train-auc:0.71029	test-auc:nan
[2]	train-auc:0.74841	test-auc:nan
[3]	train-auc:0.76300	test-auc:nan
[4]	train-auc:0.77527	test-auc:nan
[5]	train-auc:0.78598	test-auc:nan
[6]	train-auc:0.79742	test-auc:nan
[7]	train-auc:0.80023	test-auc:nan
[8]	train-auc:0.80449	test-auc:nan
[9]	train-auc:0.80986	test-auc:nan
[10]	train-auc:0.81468	test-auc:nan
[11]	train-auc:0.81834	test-auc:nan
[12]	train-auc:0.82362	test-auc:nan
[13]	train-auc:0.82832	test-auc:nan
[14]	train-auc:0.82917	test-auc:nan
[15]	train-auc:0.83412	test-auc:nan
[16]	train-auc:0.83691	test-auc:nan
[17]	train-auc:0.84106	test-auc:nan
[18]	train-auc:0.84392	test-auc:nan
[19]	train-auc:0.84750	test-auc:nan
[20]	train-auc:0.85005	test-auc:nan
[21]	train-auc:0.85199	test-auc:nan
[22]	train-auc:0.85675	test-auc:nan
[23]	train-auc:0.85933	test-auc:nan
[24]	train-auc:0.86200	test-auc:nan
[25]	train-auc:0.86485	test-auc:nan
[26]	train-auc:0.86643	test-auc:nan
[27]	train-auc:0.86785	test-auc:nan
[2

c:\Users\carte\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\carte\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:19:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\metric\auc.cc:320: Dataset is empty, or contains only positive or negative samples.
  warnings.warn(smsg, UserWarning)


In [52]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
# teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Full Model Scores"] = scores

print(teams)

           B_Team         R_Team  Small Model Scores (0.13)  \
0   Team Vitality  Team Heretics                   0.633670   
1          Fnatic          Rogue                   0.651725   
2        Team BDS     G2 Esports                   0.640879   
3    Karmine Corp         GIANTX                   0.586876   
4   MAD Lions KOI      SK Gaming                   0.668630   
5   Team Vitality       Team BDS                   0.526131   
6   Team Heretics          Rogue                   0.628600   
7          Fnatic   Karmine Corp                   0.628128   
8          GIANTX      SK Gaming                   0.611577   
9   MAD Lions KOI     G2 Esports                   0.518184   
10          Rogue       Team BDS                   0.447782   
11      SK Gaming  Team Vitality                   0.549669   
12         GIANTX  MAD Lions KOI                   0.504997   
13  Team Heretics         Fnatic                   0.537114   
14     G2 Esports   Karmine Corp                   0.62

In [53]:
teams.to_csv("predictions/shitModel.csv")